In [1]:
from typing import Callable

In [4]:
unary: set = {"--"}
binary: set = {"+", "-", "*", "/"}


def simple_rpn_to_infix(expression: list):
    global unary, binary
    stack = []

    for token in expression:
        if token.isdigit():
            # Push number onto the stack
            stack.append(token)
            continue
        if token in binary:
            operand2 = stack.pop()
            operand1 = stack.pop()
            stack.append(f"({operand1} {token} {operand2})")
            continue
        if token in unary:
            operand = stack.pop()
            stack.append(f"({token}{operand})")

    return stack[0]



In [5]:
def unary_lambda(symbol: str) -> Callable[[str], str]:
    return lambda a: f"({a}){symbol}"


letters = "abcdefghijklmnopqrstuvwxyz"
digits = "0123456789"

quantifiers: set[str] = {"*", "+", "?"}

unary_regex: dict[str, Callable[[str], str]] = {
    **{symbol: unary_lambda(symbol) for symbol in quantifiers},
    **{symbol: unary_lambda(f"{symbol}?") for symbol in quantifiers},  # greedy versions
}
binary_regex: dict[str, Callable[[str, str], str]] = {
    "concat": lambda a, b: a + b,  # concatenation
    "|": lambda a, b: f"{a}|{b}",
}
many_to_one_regex: dict[str, Callable[[list[str]], str]] = {
    "[]": lambda a: "[" + "".join([f"({x})" for x in a]) + "]"  # set
}

specials_regex: set[str] = {    ".",
    "^",
    "$",}

operands_regex: set[str] = {
*specials_regex,
*[f"\\{s}" for s in specials_regex],
    *letters,
    *letters.upper(),
    *digits,
    *[f"\\{s}" for s in quantifiers],
}


def rpn_to_infix_regex(expression: list):
    global unary_regex, binary_regex, operands_regex, many_to_one_regex
    stack = []

    for token in expression:
        if token in many_to_one_regex:
            stack = [many_to_one_regex[token](list(reversed(stack)))]
            continue

        if token in binary_regex:
            operand2 = stack.pop()
            operand1 = stack.pop()
            stack.append(binary_regex[token](operand1, operand2))
            continue
        if token in unary_regex:
            operand = stack.pop()
            stack.append(f"({token}{operand})")
            continue
        if token in operands_regex:
            stack.append(token)
            continue
        raise RuntimeError(f"Operand '{token}' is unknown")

    return stack[0]


rpn_expression_regex = ["a", "b", "c", "concat", "[]"]
infix_expression_regex = rpn_to_infix_regex(rpn_expression_regex)

In [6]:
actions = operands_regex
actions =actions.union(set(binary_regex.keys()))
actions =actions.union(set(unary_regex.keys()))
actions =actions.union(set(many_to_one_regex.keys()))

In [7]:
import re

In [ ]:
class Environment:
    def __init__(self, text, target, action_space):
        self.text = text
        self.target = target
        self.action_space = action_space

    def get_reward(self, regexp):
        array =  [x.span() for x in re.finditer(regexp, self.text)]
        bit_mask = [0 for _ in range(len(self.text))]

        for it in  array:
            for i in range(it[0], it[1]):
                bit_mask[i] = 1


        



In [26]:
env = Environment("123   78 ", "123", 10)

env.get_reward("[1-9]")

[1, 1, 1, 0, 0, 0, 1, 1, 0]